In [1]:
from numpy import *
from numpy import linalg as la
#import svdRec
def loadExData():
    return
def ecludSim(inA,inB):
    return 1.0/(1.0 + la.norm(inA - inB))
def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

data=[[0, 0, 0, 2, 2],
      [0, 0, 0, 3, 3],
      [0, 0, 0, 1, 1],
      [1, 1, 1, 0, 0],
      [2, 2, 2, 0, 0],
      [5, 5, 5, 0, 0],
      [1, 1, 1, 0, 0]]
myMat=mat(data)
print("cosine similarity")
print(cosSim(myMat[:,0],myMat[:,4]))
print(cosSim(myMat[:,0],myMat[:,0]))
print("Euler similarity")
print(ecludSim(myMat[:,0],myMat[:,4]))
print(ecludSim(myMat[:,0],myMat[:,0]))
print("pearson similarity")
print(pearsSim(myMat[:,0],myMat[:,4]))
print(pearsSim(myMat[:,0],myMat[:,0]))

cosine similarity
0.5
1.0
Euler similarity
0.12973190755680383
1.0
pearson similarity
0.20596538173840329
1.0


In [2]:
import pandas as pd
#读入数据集--u.data文件，查看用户/项目排名信息

heads = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=heads)
print(ratings)

       user_id  item_id  rating  timestamp
0          196      242       3  881250949
1          186      302       3  891717742
2           22      377       1  878887116
3          244       51       2  880606923
4          166      346       1  886397596
5          298      474       4  884182806
6          115      265       2  881171488
7          253      465       5  891628467
8          305      451       3  886324817
9            6       86       3  883603013
10          62      257       2  879372434
11         286     1014       5  879781125
12         200      222       5  876042340
13         210       40       3  891035994
14         224       29       3  888104457
15         303      785       3  879485318
16         122      387       5  879270459
17         194      274       2  879539794
18         291     1042       4  874834944
19         234     1184       2  892079237
20         119      392       4  886176814
21         167      486       4  892738452
22         

In [3]:
print("len of users:",len(ratings))

len of users: 100000


In [4]:
import pandas as pd
# 读入数据
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')
print(users)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')
print(ratings)

m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url'] 
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),encoding='latin-1') 
print(movies)

     user_id  age sex     occupation zip_code
0          1   24   M     technician    85711
1          2   53   F          other    94043
2          3   23   M         writer    32067
3          4   24   M     technician    43537
4          5   33   F          other    15213
5          6   42   M      executive    98101
6          7   57   M  administrator    91344
7          8   36   M  administrator    05201
8          9   29   M        student    01002
9         10   53   M         lawyer    90703
10        11   39   F          other    30329
11        12   28   F          other    06405
12        13   47   M       educator    29206
13        14   45   M      scientist    55106
14        15   49   F       educator    97301
15        16   21   M  entertainment    10309
16        17   30   M     programmer    06355
17        18   35   F          other    37212
18        19   40   M      librarian    02138
19        20   42   F      homemaker    95660
20        21   26   M         writ

In [6]:
import numpy as np
import pandas as pd
#from sklearn import cross_validation as cv
import sklearn.model_selection as cv
from sklearn.metrics.pairwise import pairwise_distances

#CF预测算法
def predict(ratings, similarity, type='user'):
    #user-based CF
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    #item-based CF
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

#算法性能评估
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

#1.读入数据集--u.data文件
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=header)
   #计算种类（不重复的用户/电影数）
n_users = df.user_id.unique().shape[0]
n_movies = df.item_id.unique().shape[0]
print('Number of users = ',str(n_users), ' | Number of movies = ' , str(n_movies)  )

#2.拆分成测试集和训练集
train_data, test_data = cv.train_test_split(df, test_size=0.25)

#3.创建两个用户/电影矩阵：一个测试数据矩阵，一个训练数据矩阵。
train_matrix = np.zeros((n_users, n_movies))
for line in train_data.itertuples():
    train_matrix[line[1]-1, line[2]-1] = line[3]  

test_matrix = np.zeros((n_users, n_movies))
for line in test_data.itertuples():
    test_matrix[line[1]-1, line[2]-1] = line[3]
    
#4.计算余弦相似性。pairwise_distances函数输出范围0~1。    
user_sim = pairwise_distances(train_matrix, metric='cosine')
movie_sim = pairwise_distances(train_matrix.T, metric='cosine')#T转置，转变计算方向

#5.CF预测
movie_predict = predict(train_matrix, movie_sim, type='item')
user_predict = predict(train_matrix, user_sim, type='user')

##6.评估
print('U-based MSE = ', str(rmse(user_predict, test_matrix)))
print('M-based MSE= ', str(rmse(movie_predict, test_matrix)))

Number of users =  943  | Number of movies =  1682
U-based MSE =  3.130839804301856
M-based MSE=  3.4575750661792646
